In [1]:
import numpy as np
import pandas as pd
import xarray as xr

Design data query di  data GLCT trans (awali dg membubuhi country grids) utk menjawab:
Kelas GLCT agri berasal dari GLTC class apa sblmnya? Akumulasi 1970-2020
Pada masing2 region, bagaimana statistiknya (agri berasal dari kelas apa saja?)

In [2]:
glct_trans = xr.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GLCT_trans.nc", engine="netcdf4")

In [3]:
country_code = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/ISO-3166-Country-Code_Final.xlsx")
luh_static = xr.open_dataset("D:/kerja/asisten riset/code v/step 3/GbGlct/gpw_v4_national_identifier_grid_rev11_5_min_finall.nc", engine="netcdf4")
country_code.head()

,ISO Country,alpha-2,alpha-3,country-code,iso_3166-2,Sub Region,ISO Region,IMAGE Region Code,IMAGE Region Name
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,South Asia,Asia,RSAS,Rest S. Asia
1,Albania,AL,ALB,8,ISO 3166-2:AL,Europe and Russia,Europe,CEU,C. Europe
2,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Middle East,Africa,NAF,N. Africa
3,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Oceania,OCE,Oceania
4,Andorra,AD,AND,20,ISO 3166-2:AD,Europe and Russia,Europe,WEU,W. Europe


In [4]:
ccode_iso = list(country_code['country-code'])
cname_iso = list(country_code['ISO Country'])

ccode_dict = {}
for i, ccode in enumerate(ccode_iso):
    ccode_dict[ccode] = cname_iso[i]

ccode_worldwide_int = luh_static['ccode'].to_numpy().astype('int64')

ccode_convert = np.zeros((2160, 4320), dtype="<U14") #rubah ke <U64

for i in range(2160):
    for j in range(4320):
        if (ccode_worldwide_int[i][j] in ccode_dict.keys()):
            ccode_convert[i][j] = ccode_dict[ccode_worldwide_int[i][j]]
        else:
            ccode_convert[i][j] = "ocean"
            
country = xr.Dataset({"country": (["latitude", "longitude"], ccode_convert)},
                         coords={ "longitude": glct_trans.coords["longitude"].to_numpy(), "latitude": glct_trans.coords["latitude"].to_numpy()})

In [5]:
glct5mins_trans = np.zeros((10, 2160, 4320), dtype='object')
for i in range(10):
    prev_nicknames_with_to = np.add(glct_trans['GLCT_trans'].isel(time=i), '_to_')
    glct5mins_trans[i] = np.add(prev_nicknames_with_to, glct_trans['GLCT_trans'].isel(time=i+1))

In [6]:
glct_trans_netcdf = xr.Dataset({
        "GLCT_trans":(["time", "latitude", "longitude"], glct5mins_trans)
    },
    coords={
        "time": pd.date_range(start='1975-01-01', end='2020-01-01', freq='5YS'),
        "latitude": glct_trans.coords["latitude"].to_numpy(),
        "longitude": glct_trans.coords["longitude"].to_numpy(),
        # "country": country['country']
    })
glct_trans_netcdf

<xarray.Dataset>
Dimensions:     (time: 2, latitude: 2160, longitude: 4320)
Coordinates:
  * time        (time) datetime64[ns] 1975-01-01 1980-01-01
  * latitude    (latitude) float64 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
  * longitude   (longitude) float64 -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
Data variables:
    GLCT_trans  (time, latitude, longitude) object 'ocean_to_ocean' ... 'ocea...

In [12]:
glct_trans_netcdf.to_netcdf("D:/kerja/asisten riset/code v/step 3/GbGlct/GLCT_trans_to.nc", mode="w", format="NETCDF4")

bore_to_agri
ccfo_to_agri
dsrt_to_agri
rfab_to_agri
rftm_to_agri
scrb_to_agri
stepp_to_agri
svna_to_agri
tdfo_to_agri
tmfo_to_agri
trof_to_agri
trow_to_agri
tund_to_agri
tuwd_to_agri
wmfo_to_agri

In [20]:
glct5mins_trans_filter = np.zeros((1, 2160, 4320), dtype='object')
glct5mins_trans_filter[0] = np.where(
        (glct_trans_netcdf['GLCT_trans'].isel(time=0) == 'bore_to_agri')|
        (glct_trans_netcdf['GLCT_trans'].isel(time=0) == 'ccfo_to_agri')|
        (glct_trans_netcdf['GLCT_trans'].isel(time=0) == 'dsrt_to_agri')|
        (glct_trans_netcdf['GLCT_trans'].isel(time=0) == 'rfab_to_agri')|
        (glct_trans_netcdf['GLCT_trans'].isel(time=0) == 'rftm_to_agri')|
        (glct_trans_netcdf['GLCT_trans'].isel(time=0) == 'scrb_to_agri')|
        (glct_trans_netcdf['GLCT_trans'].isel(time=0) == 'stepp_to_agri')|
        (glct_trans_netcdf['GLCT_trans'].isel(time=0) == 'svna_to_agri')|
        (glct_trans_netcdf['GLCT_trans'].isel(time=0) == 'tdfo_to_agri')|
        (glct_trans_netcdf['GLCT_trans'].isel(time=0) == 'tmfo_to_agri')|
        (glct_trans_netcdf['GLCT_trans'].isel(time=0) == 'trof_to_agri')|
        (glct_trans_netcdf['GLCT_trans'].isel(time=0) == 'trow_to_agri')|
        (glct_trans_netcdf['GLCT_trans'].isel(time=0) == 'tund_to_agri')|
        (glct_trans_netcdf['GLCT_trans'].isel(time=0) == 'tuwd_to_agri')|
        (glct_trans_netcdf['GLCT_trans'].isel(time=0) == 'wmfo_to_agri'), glct_trans_netcdf['GLCT_trans'].isel(time=0).values, '')

In [22]:
glct_trans_filter_netcdf = xr.Dataset({
        "GLCT_trans":(["time", "latitude", "longitude"], glct5mins_trans_filter)
    },
    coords={
        "time": pd.to_datetime(['1975-01-01']),
        "latitude": glct_trans.coords["latitude"].to_numpy(),
        "longitude": glct_trans.coords["longitude"].to_numpy(),
        "country": country['country']
    })
glct_trans_filter_netcdf

<xarray.Dataset>
Dimensions:     (time: 1, latitude: 2160, longitude: 4320)
Coordinates:
  * longitude   (longitude) float64 -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
  * latitude    (latitude) float64 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
  * time        (time) datetime64[ns] 1975-01-01
    country     (latitude, longitude) <U14 'ocean' 'ocean' ... 'ocean' 'ocean'
Data variables:
    GLCT_trans  (time, latitude, longitude) object '' '' '' '' ... '' '' '' ''

In [29]:
df = glct_trans_filter_netcdf.isel(time=0).to_dataframe()
df.head()

GLCT_trans       time country
latitude  longitude                                
89.958333 -179.958333            1975-01-01   ocean
          -179.875000            1975-01-01   ocean
          -179.791667            1975-01-01   ocean
          -179.708333            1975-01-01   ocean
          -179.625000            1975-01-01   ocean

In [31]:
df_index = df.reset_index()
data = pd.to_datetime(df_index['time'].values, format='%d/%m/%Y %H.%M.%S')
df_index['time'] = data.year
df_index.head()

,latitude,longitude,GLCT_trans,time,country
0,89.958333,-179.958333,,1975,ocean
1,89.958333,-179.875000,,1975,ocean
2,89.958333,-179.791667,,1975,ocean
3,89.958333,-179.708333,,1975,ocean
4,89.958333,-179.625000,,1975,ocean


In [13]:
garea = xr.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis1_slidingwindows/GAREA.NC", engine="netcdf4")
garea_1975 = garea.isel(time=slice(1, 11)) #slice 1975 saja
garea_ha = np.multiply(garea_1975, 100)
garea_ha

<xarray.Dataset>
Dimensions:    (time: 2, latitude: 2160, longitude: 4320)
Coordinates:
  * longitude  (longitude) float64 -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
  * latitude   (latitude) float64 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
  * time       (time) datetime64[ns] 1975-01-01 1980-01-01
Data variables:
    GAREA      (time, latitude, longitude) float32 nan nan nan ... nan nan nan
Attributes:
    modelname:    IMAGE_LPJ
    modelinfo:    image25/MODEL/trunk/model#2844,interface_imagelpj/trunk#170,#0
    conventions:  CF-1.6
    institution:  PBL netherlands Environmental Assessment Agency ( http//www...
    source:       image3.2,interface3.2,lpj3.2
    references:   http//www.pbl.nl/image
    disclaimer:   http//themasites.pbl.nl/models/image/index.php/IMAGE-rights

In [44]:
df_value = garea_ha.isel(time=0).to_dataframe()
df_value = df_value.reset_index()
data = pd.to_datetime(df_value['time'].values, format='%d/%m/%Y %H.%M.%S')
df_value['time'] = data.year
df_value.head()

,longitude,latitude,time,GAREA
0,-179.958333,89.958333,1975,NaN
1,-179.958333,89.875000,1975,NaN
2,-179.958333,89.791667,1975,NaN
3,-179.958333,89.708333,1975,NaN
4,-179.958333,89.625000,1975,NaN


In [39]:
df_merge = pd.merge(left=df_index, right=df_value, left_on=['latitude',	'longitude', 'time'], right_on=['latitude',	'longitude', 'time'])
df_merge.head()

,latitude,longitude,GLCT_trans,time,country,GAREA
0,89.958333,-179.958333,,1975,ocean,NaN
1,89.958333,-179.875000,,1975,ocean,NaN
2,89.958333,-179.791667,,1975,ocean,NaN
3,89.958333,-179.708333,,1975,ocean,NaN
4,89.958333,-179.625000,,1975,ocean,NaN


In [48]:
df_merge_filter = df_merge[df_merge['GLCT_trans'] != ""]
df_merge_filter

,latitude,longitude,GLCT_trans,time,country,GAREA
1376246,63.458333,27.208333,ccfo_to_agri,1975,Finland,3315.959961
1376253,63.458333,27.791666,ccfo_to_agri,1975,Finland,3552.610107
1380572,63.375000,27.708333,ccfo_to_agri,1975,Finland,3847.959961
1380573,63.375000,27.791666,ccfo_to_agri,1975,Finland,3762.489990
1384889,63.291667,27.458333,ccfo_to_agri,1975,Finland,3620.850098
...,...,...,...,...,...,...
7466255,-54.041666,-72.041667,bore_to_agri,1975,Chile,4855.220215
7466296,-54.041666,-68.625000,bore_to_agri,1975,Argentina,5041.859863
7479250,-54.291666,-69.125000,bore_to_agri,1975,Chile,5011.489746
7487894,-54.458333,-68.791667,bore_to_agri,1975,Chile,4991.189941


In [53]:
df_merge_columns = df_merge_filter[df_merge_filter.columns[2:]]

In [54]:
df_merge_columns_merge = pd.merge(left=df_merge_columns, right=country_code, left_on=['country'], right_on=['ISO Country'])
df_merge_columns_merge.head()

,GLCT_trans,time,country,GAREA,ISO Country,alpha-2,alpha-3,country-code,iso_3166-2,Sub Region,ISO Region,IMAGE Region Code,IMAGE Region Name
0,ccfo_to_agri,1975,Finland,3315.959961,Finland,FI,FIN,246,ISO 3166-2:FI,Europe and Russia,Europe,WEU,W. Europe
1,ccfo_to_agri,1975,Finland,3552.610107,Finland,FI,FIN,246,ISO 3166-2:FI,Europe and Russia,Europe,WEU,W. Europe
2,ccfo_to_agri,1975,Finland,3847.959961,Finland,FI,FIN,246,ISO 3166-2:FI,Europe and Russia,Europe,WEU,W. Europe
3,ccfo_to_agri,1975,Finland,3762.489990,Finland,FI,FIN,246,ISO 3166-2:FI,Europe and Russia,Europe,WEU,W. Europe
4,ccfo_to_agri,1975,Finland,3620.850098,Finland,FI,FIN,246,ISO 3166-2:FI,Europe and Russia,Europe,WEU,W. Europe


In [56]:
df_merge_columns_merge = df_merge_columns_merge[['GLCT_trans', 'time','GAREA','IMAGE Region Name']]
df_merge_columns_merge.head()

,GLCT_trans,time,GAREA,IMAGE Region Name
0,ccfo_to_agri,1975,3315.959961,W. Europe
1,ccfo_to_agri,1975,3552.610107,W. Europe
2,ccfo_to_agri,1975,3847.959961,W. Europe
3,ccfo_to_agri,1975,3762.489990,W. Europe
4,ccfo_to_agri,1975,3620.850098,W. Europe


In [62]:
table = pd.pivot_table(df_merge_columns_merge, values=["GAREA"], index=["IMAGE Region Name", "GLCT_trans"], columns=['time'], fill_value=0, aggfunc="sum")
df_table = table.stack(level=0, future_stack=True)
df_table = df_table.reset_index()
df_table.drop(columns='level_2', inplace=True)
df_table.head()

time,IMAGE Region Name,GLCT_trans,1975
0,Brazil,scrb_to_agri,5.273259e+05
1,Brazil,stepp_to_agri,8.475680e+03
2,Brazil,svna_to_agri,1.184868e+07
3,Brazil,trof_to_agri,1.261181e+06
4,Brazil,trow_to_agri,3.150911e+06


--------------------------------------------------------

In [7]:
glct5mins_trans_filter = np.zeros((10, 2160, 4320), dtype='object')

max_year = 10
for i in range(max_year): #1975
    glct5mins_trans_filter[i] = np.where(
    (glct_trans_netcdf['GLCT_trans'].isel(time=i) == 'bore_to_agri')|
    (glct_trans_netcdf['GLCT_trans'].isel(time=i) == 'ccfo_to_agri')|
    (glct_trans_netcdf['GLCT_trans'].isel(time=i) == 'dsrt_to_agri')|
    (glct_trans_netcdf['GLCT_trans'].isel(time=i) == 'rfab_to_agri')|
    (glct_trans_netcdf['GLCT_trans'].isel(time=i) == 'rftm_to_agri')|
    (glct_trans_netcdf['GLCT_trans'].isel(time=i) == 'scrb_to_agri')|
    (glct_trans_netcdf['GLCT_trans'].isel(time=i) == 'stepp_to_agri')|
    (glct_trans_netcdf['GLCT_trans'].isel(time=i) == 'svna_to_agri')|
    (glct_trans_netcdf['GLCT_trans'].isel(time=i) == 'tdfo_to_agri')|
    (glct_trans_netcdf['GLCT_trans'].isel(time=i) == 'tmfo_to_agri')|
    (glct_trans_netcdf['GLCT_trans'].isel(time=i) == 'trof_to_agri')|
    (glct_trans_netcdf['GLCT_trans'].isel(time=i) == 'trow_to_agri')|
    (glct_trans_netcdf['GLCT_trans'].isel(time=i) == 'tund_to_agri')|
    (glct_trans_netcdf['GLCT_trans'].isel(time=i) == 'tuwd_to_agri')|
    (glct_trans_netcdf['GLCT_trans'].isel(time=i) == 'wmfo_to_agri'), glct_trans_netcdf['GLCT_trans'].isel(time=i).values, '')

In [8]:
glct_trans_filter_netcdf = xr.Dataset({
        "GLCT_trans":(["time", "latitude", "longitude"], glct5mins_trans_filter[0:2])
    },
    coords={
        "time": pd.date_range(start='1975-01-01', end='2020-01-01', freq='5YS'),
        "latitude": glct_trans.coords["latitude"].to_numpy(),
        "longitude": glct_trans.coords["longitude"].to_numpy(),
        "country": country['country']
    })
glct_trans_filter_netcdf

<xarray.Dataset>
Dimensions:     (time: 2, latitude: 2160, longitude: 4320)
Coordinates:
  * longitude   (longitude) float64 -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
  * latitude    (latitude) float64 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
  * time        (time) datetime64[ns] 1975-01-01 1980-01-01
    country     (latitude, longitude) <U14 'ocean' 'ocean' ... 'ocean' 'ocean'
Data variables:
    GLCT_trans  (time, latitude, longitude) object '' '' '' '' ... '' '' '' ''

In [9]:
df = glct_trans_filter_netcdf.isel(time=0).to_dataframe()
df1 = glct_trans_filter_netcdf.isel(time=1).to_dataframe()
df2 = glct_trans_filter_netcdf.isel(time=2).to_dataframe()
df3 = glct_trans_filter_netcdf.isel(time=3).to_dataframe()
df4 = glct_trans_filter_netcdf.isel(time=4).to_dataframe()
df5 = glct_trans_filter_netcdf.isel(time=5).to_dataframe()
df6 = glct_trans_filter_netcdf.isel(time=6).to_dataframe()
df7 = glct_trans_filter_netcdf.isel(time=7).to_dataframe()
df8 = glct_trans_filter_netcdf.isel(time=8).to_dataframe()
df9 = glct_trans_filter_netcdf.isel(time=9).to_dataframe()

In [10]:
df_index = df.reset_index()
df_index1 = df1.reset_index()
df_index2 = df2.reset_index()
df_index3 = df3.reset_index()
df_index4 = df4.reset_index()
df_index5 = df5.reset_index()
df_index6 = df6.reset_index()
df_index7 = df7.reset_index()
df_index8 = df8.reset_index()
df_index9 = df9.reset_index()

data = pd.to_datetime(df_index['time'].values, format='%d/%m/%Y %H.%M.%S')
df_index['time'] = data.year

data1 = pd.to_datetime(df_index1['time'].values, format='%d/%m/%Y %H.%M.%S')
df_index1['time'] = data1.year

data2 = pd.to_datetime(df_index2['time'].values, format='%d/%m/%Y %H.%M.%S')
df_index2['time'] = data2.year

data3 = pd.to_datetime(df_index3['time'].values, format='%d/%m/%Y %H.%M.%S')
df_index3['time'] = data3.year

data4 = pd.to_datetime(df_index4['time'].values, format='%d/%m/%Y %H.%M.%S')
df_index4['time'] = data4.year

data5 = pd.to_datetime(df_index5['time'].values, format='%d/%m/%Y %H.%M.%S')
df_index5['time'] = data5.year

data6 = pd.to_datetime(df_index6['time'].values, format='%d/%m/%Y %H.%M.%S')
df_index6['time'] = data6.year

data7 = pd.to_datetime(df_index7['time'].values, format='%d/%m/%Y %H.%M.%S')
df_index7['time'] = data7.year

data8 = pd.to_datetime(df_index8['time'].values, format='%d/%m/%Y %H.%M.%S')
df_index8['time'] = data8.year

data9 = pd.to_datetime(df_index9['time'].values, format='%d/%m/%Y %H.%M.%S')
df_index9['time'] = data9.year


In [14]:
df_value = garea_ha.isel(time=0).to_dataframe()
df_value1 = garea_ha.isel(time=1).to_dataframe()
df_value2 = garea_ha.isel(time=2).to_dataframe()
df_value3 = garea_ha.isel(time=3).to_dataframe()
df_value4 = garea_ha.isel(time=4).to_dataframe()
df_value5 = garea_ha.isel(time=5).to_dataframe()
df_value6 = garea_ha.isel(time=6).to_dataframe()
df_value7 = garea_ha.isel(time=7).to_dataframe()
df_value8 = garea_ha.isel(time=8).to_dataframe()
df_value9 = garea_ha.isel(time=9).to_dataframe()

df_value = df_value.reset_index()
df_value1 = df_value1.reset_index()
df_value2 = df_value2.reset_index()
df_value3 = df_value3.reset_index()
df_value4 = df_value4.reset_index()
df_value5 = df_value5.reset_index()
df_value6 = df_value6.reset_index()
df_value7 = df_value7.reset_index()
df_value8 = df_value8.reset_index()
df_value9 = df_value9.reset_index()

data = pd.to_datetime(df_value['time'].values, format='%d/%m/%Y %H.%M.%S')
df_value['time'] = data.year

data1 = pd.to_datetime(df_value1['time'].values, format='%d/%m/%Y %H.%M.%S')
df_value1['time'] = data1.year

data2 = pd.to_datetime(df_value2['time'].values, format='%d/%m/%Y %H.%M.%S')
df_value2['time'] = data2.year

data3 = pd.to_datetime(df_value3['time'].values, format='%d/%m/%Y %H.%M.%S')
df_value3['time'] = data3.year

data4 = pd.to_datetime(df_value4['time'].values, format='%d/%m/%Y %H.%M.%S')
df_value4['time'] = data4.year

data5 = pd.to_datetime(df_value5['time'].values, format='%d/%m/%Y %H.%M.%S')
df_value5['time'] = data5.year

data6 = pd.to_datetime(df_value6['time'].values, format='%d/%m/%Y %H.%M.%S')
df_value6['time'] = data6.year

data7 = pd.to_datetime(df_value7['time'].values, format='%d/%m/%Y %H.%M.%S')
df_value7['time'] = data7.year

data8 = pd.to_datetime(df_value8['time'].values, format='%d/%m/%Y %H.%M.%S')
df_value8['time'] = data8.year

data9 = pd.to_datetime(df_value9['time'].values, format='%d/%m/%Y %H.%M.%S')
df_value9['time'] = data9.year



In [15]:
df_merge = pd.merge(left=df_index, right=df_value, left_on=['latitude',	'longitude', 'time'], right_on=['latitude',	'longitude', 'time'])
df_merge1 = pd.merge(left=df_index1, right=df_value1, left_on=['latitude',	'longitude', 'time'], right_on=['latitude',	'longitude', 'time'])
df_merge2 = pd.merge(left=df_index2, right=df_value2, left_on=['latitude',	'longitude', 'time'], right_on=['latitude',	'longitude', 'time'])
df_merge3 = pd.merge(left=df_index3, right=df_value3, left_on=['latitude',	'longitude', 'time'], right_on=['latitude',	'longitude', 'time'])
df_merge4 = pd.merge(left=df_index4, right=df_value4, left_on=['latitude',	'longitude', 'time'], right_on=['latitude',	'longitude', 'time'])
df_merge5 = pd.merge(left=df_index5, right=df_value5, left_on=['latitude',	'longitude', 'time'], right_on=['latitude',	'longitude', 'time'])
df_merge6 = pd.merge(left=df_index6, right=df_value6, left_on=['latitude',	'longitude', 'time'], right_on=['latitude',	'longitude', 'time'])
df_merge7 = pd.merge(left=df_index7, right=df_value7, left_on=['latitude',	'longitude', 'time'], right_on=['latitude',	'longitude', 'time'])
df_merge8 = pd.merge(left=df_index8, right=df_value8, left_on=['latitude',	'longitude', 'time'], right_on=['latitude',	'longitude', 'time'])
df_merge9 = pd.merge(left=df_index9, right=df_value9, left_on=['latitude',	'longitude', 'time'], right_on=['latitude',	'longitude', 'time'])

In [25]:
df_merge_filter = df_merge[df_merge['GLCT_trans'] != ""]
df_merge_filter1 = df_merge1[df_merge1['GLCT_trans'] != ""]
df_merge_filter2 = df_merge2[df_merge2['GLCT_trans'] != ""]
df_merge_filter3 = df_merge3[df_merge3['GLCT_trans'] != ""]
df_merge_filter4 = df_merge4[df_merge4['GLCT_trans'] != ""]
df_merge_filter5 = df_merge5[df_merge5['GLCT_trans'] != ""]
df_merge_filter6 = df_merge6[df_merge6['GLCT_trans'] != ""]
df_merge_filter7 = df_merge7[df_merge7['GLCT_trans'] != ""]
df_merge_filter8 = df_merge8[df_merge8['GLCT_trans'] != ""]
df_merge_filter9 = df_merge9[df_merge9['GLCT_trans'] != ""]

#filter lat, lon
df_merge_columns = df_merge_filter[df_merge_filter.columns[2:]]
df_merge_columns1 = df_merge_filter1[df_merge_filter1.columns[2:]]
df_merge_columns2 = df_merge_filter2[df_merge_filter2.columns[2:]]
df_merge_columns3 = df_merge_filter3[df_merge_filter3.columns[2:]]
df_merge_columns4 = df_merge_filter4[df_merge_filter4.columns[2:]]
df_merge_columns5 = df_merge_filter5[df_merge_filter5.columns[2:]]
df_merge_columns6 = df_merge_filter6[df_merge_filter6.columns[2:]]
df_merge_columns7 = df_merge_filter7[df_merge_filter7.columns[2:]]
df_merge_columns8 = df_merge_filter8[df_merge_filter8.columns[2:]]
df_merge_columns9 = df_merge_filter9[df_merge_filter9.columns[2:]]

In [26]:
df_merge_columns_merge = pd.merge(left=df_merge_columns, right=country_code, left_on=['country'], right_on=['ISO Country'])
df_merge_columns_merge1 = pd.merge(left=df_merge_columns1, right=country_code, left_on=['country'], right_on=['ISO Country'])
df_merge_columns_merge2 = pd.merge(left=df_merge_columns2, right=country_code, left_on=['country'], right_on=['ISO Country'])
df_merge_columns_merge3 = pd.merge(left=df_merge_columns3, right=country_code, left_on=['country'], right_on=['ISO Country'])
df_merge_columns_merge4 = pd.merge(left=df_merge_columns4, right=country_code, left_on=['country'], right_on=['ISO Country'])
df_merge_columns_merge5 = pd.merge(left=df_merge_columns5, right=country_code, left_on=['country'], right_on=['ISO Country'])
df_merge_columns_merge6 = pd.merge(left=df_merge_columns6, right=country_code, left_on=['country'], right_on=['ISO Country'])
df_merge_columns_merge7 = pd.merge(left=df_merge_columns7, right=country_code, left_on=['country'], right_on=['ISO Country'])
df_merge_columns_merge8 = pd.merge(left=df_merge_columns8, right=country_code, left_on=['country'], right_on=['ISO Country'])
df_merge_columns_merge9 = pd.merge(left=df_merge_columns9, right=country_code, left_on=['country'], right_on=['ISO Country'])

In [27]:
df_merge_columns_merge = df_merge_columns_merge[['GLCT_trans', 'time','GAREA','IMAGE Region Name']]
df_merge_columns_merge1 = df_merge_columns_merge1[['GLCT_trans', 'time','GAREA','IMAGE Region Name']]
df_merge_columns_merge2 = df_merge_columns_merge2[['GLCT_trans', 'time','GAREA','IMAGE Region Name']]
df_merge_columns_merge3 = df_merge_columns_merge3[['GLCT_trans', 'time','GAREA','IMAGE Region Name']]
df_merge_columns_merge4 = df_merge_columns_merge4[['GLCT_trans', 'time','GAREA','IMAGE Region Name']]
df_merge_columns_merge5 = df_merge_columns_merge5[['GLCT_trans', 'time','GAREA','IMAGE Region Name']]
df_merge_columns_merge6 = df_merge_columns_merge6[['GLCT_trans', 'time','GAREA','IMAGE Region Name']]
df_merge_columns_merge7 = df_merge_columns_merge7[['GLCT_trans', 'time','GAREA','IMAGE Region Name']]
df_merge_columns_merge8 = df_merge_columns_merge8[['GLCT_trans', 'time','GAREA','IMAGE Region Name']]
df_merge_columns_merge9 = df_merge_columns_merge9[['GLCT_trans', 'time','GAREA','IMAGE Region Name']]

In [28]:
table = pd.pivot_table(df_merge_columns_merge, values=["GAREA"], index=["IMAGE Region Name", "GLCT_trans"], columns=['time'], fill_value=0, aggfunc="sum")
table1 = pd.pivot_table(df_merge_columns_merge1, values=["GAREA"], index=["IMAGE Region Name", "GLCT_trans"], columns=['time'], fill_value=0, aggfunc="sum")
table2 = pd.pivot_table(df_merge_columns_merge2, values=["GAREA"], index=["IMAGE Region Name", "GLCT_trans"], columns=['time'], fill_value=0, aggfunc="sum")
table3 = pd.pivot_table(df_merge_columns_merge3, values=["GAREA"], index=["IMAGE Region Name", "GLCT_trans"], columns=['time'], fill_value=0, aggfunc="sum")
table4 = pd.pivot_table(df_merge_columns_merge4, values=["GAREA"], index=["IMAGE Region Name", "GLCT_trans"], columns=['time'], fill_value=0, aggfunc="sum")
table5 = pd.pivot_table(df_merge_columns_merge5, values=["GAREA"], index=["IMAGE Region Name", "GLCT_trans"], columns=['time'], fill_value=0, aggfunc="sum")
table6 = pd.pivot_table(df_merge_columns_merge6, values=["GAREA"], index=["IMAGE Region Name", "GLCT_trans"], columns=['time'], fill_value=0, aggfunc="sum")
table7 = pd.pivot_table(df_merge_columns_merge7, values=["GAREA"], index=["IMAGE Region Name", "GLCT_trans"], columns=['time'], fill_value=0, aggfunc="sum")
table8 = pd.pivot_table(df_merge_columns_merge8, values=["GAREA"], index=["IMAGE Region Name", "GLCT_trans"], columns=['time'], fill_value=0, aggfunc="sum")
table9 = pd.pivot_table(df_merge_columns_merge9, values=["GAREA"], index=["IMAGE Region Name", "GLCT_trans"], columns=['time'], fill_value=0, aggfunc="sum")

df_table = table.stack(level=0, future_stack=True)
df_table1 = table1.stack(level=0, future_stack=True)
df_table2 = table2.stack(level=0, future_stack=True)
df_table3 = table3.stack(level=0, future_stack=True)
df_table4 = table4.stack(level=0, future_stack=True)
df_table5 = table5.stack(level=0, future_stack=True)
df_table6 = table6.stack(level=0, future_stack=True)
df_table7 = table7.stack(level=0, future_stack=True)
df_table8 = table8.stack(level=0, future_stack=True)
df_table9 = table9.stack(level=0, future_stack=True)

df_table = df_table.reset_index()
df_table1 = df_table1.reset_index()
df_table2 = df_table2.reset_index()
df_table3 = df_table3.reset_index()
df_table4 = df_table4.reset_index()
df_table5 = df_table5.reset_index()
df_table6 = df_table6.reset_index()
df_table7 = df_table7.reset_index()
df_table8 = df_table8.reset_index()
df_table9 = df_table9.reset_index()

df_table.drop(columns='level_2', inplace=True)
df_table1.drop(columns='level_2', inplace=True)
df_table2.drop(columns='level_2', inplace=True)
df_table3.drop(columns='level_2', inplace=True)
df_table4.drop(columns='level_2', inplace=True)
df_table5.drop(columns='level_2', inplace=True)
df_table6.drop(columns='level_2', inplace=True)
df_table7.drop(columns='level_2', inplace=True)
df_table8.drop(columns='level_2', inplace=True)
df_table9.drop(columns='level_2', inplace=True)

In [43]:
from functools import reduce

frames = [df_table, df_table1, df_table2, df_table3, df_table4, df_table5, df_table6, df_table7, df_table8, df_table9]


# Gabungkan semua DataFrame secara berurutan menggunakan reduce()
merged_df = reduce(lambda left, right: pd.merge(left, right, on=['IMAGE Region Name', 'GLCT_trans'], how="outer"), frames)
merged_df = merged_df.replace([np.inf, -np.inf], np.nan)
merged_df = merged_df.fillna(0)
merged_df

time,IMAGE Region Name,GLCT_trans,1975,1980
0,Brazil,rftm_to_agri,0.000000e+00,7.925550e+03
1,Brazil,scrb_to_agri,5.273259e+05,3.973095e+05
2,Brazil,stepp_to_agri,8.475680e+03,1.695534e+04
3,Brazil,svna_to_agri,1.184868e+07,8.018792e+06
4,Brazil,trof_to_agri,1.261181e+06,6.865724e+05
...,...,...,...,...
134,W. Europe,scrb_to_agri,1.222101e+05,0.000000e+00
135,W. Europe,stepp_to_agri,1.952483e+04,0.000000e+00
136,W. Europe,tdfo_to_agri,5.694662e+05,5.731440e+03
137,W. Europe,tmfo_to_agri,1.011411e+06,0.000000e+00


In [46]:
merged_df['Akumulasi'] = merged_df[1975] + merged_df[1980]
merged_df

time,IMAGE Region Name,GLCT_trans,1975,1980,Akumulasi
0,Brazil,rftm_to_agri,0.000000e+00,7.925550e+03,7.925550e+03
1,Brazil,scrb_to_agri,5.273259e+05,3.973095e+05,9.246354e+05
2,Brazil,stepp_to_agri,8.475680e+03,1.695534e+04,2.543102e+04
3,Brazil,svna_to_agri,1.184868e+07,8.018792e+06,1.986747e+07
4,Brazil,trof_to_agri,1.261181e+06,6.865724e+05,1.947754e+06
...,...,...,...,...,...
134,W. Europe,scrb_to_agri,1.222101e+05,0.000000e+00,1.222101e+05
135,W. Europe,stepp_to_agri,1.952483e+04,0.000000e+00,1.952483e+04
136,W. Europe,tdfo_to_agri,5.694662e+05,5.731440e+03,5.751977e+05
137,W. Europe,tmfo_to_agri,1.011411e+06,0.000000e+00,1.011411e+06
